In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

# Список республик бывшего СССР
resp = ['Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 'Latvia', 'Lithuania',\
        'Republic of Moldova', 'Russian Federation', 'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan']

Для начала посмотрим статистику по "Ожидаемая продолжительность жизни". Возьмем информацию только для стран бывшего СССР, только за 2019 и общую для обеих полов.

In [318]:
# Фунция для стандартного выбора данных из таблиц

f = lambda x : (x.replace('<', '').replace('No data', '0').split(" ")[0])  # Если в строке есть 2 вида данных через пробел, то берем только 1

# Передаем в нее название файла, имена столбцов, название нужной нам переменной, период и условие
def get_stat(filename, nms, period, conditions={}):
    url = "./dataframes/" + filename + ".csv"
    data = pd.read_csv(url, dtype = {'country': str, 'period': int},\
                       converters = {'result':f},\
                       header=None, skiprows=[0,], names=nms)
    
    data = data[(data['country'].isin(resp)) & (data['period']==period)] # Выбираем нужные нам страны и период
    data['result'] = data['result'].astype('float')
    
    for k, v in conditions.items(): # Перебираем дополнительные условия выборки
        data = data[(data[k]==v)]
        
    return data[['country', 'result']].sort_values(by='result', ascending=False)

#pd.pivot_table(data, values = 'age', columns = 'country')

datas = {} # В этот словарь будем собирать данные по странам

In [250]:
datas['leab'] = get_stat('lifeExpectancyAtBirth',
                     ['country','period','indicator','gender','result'],                         
                     2019,
                     {'gender':'Both sexes'})
datas['leab']

,country,result
649,Estonia,78.88
72,Armenia,76.03
1153,Lithuania,75.99
1093,Latvia,75.38
168,Belarus,74.81
1069,Kyrgyzstan,74.18
1021,Kazakhstan,73.95
745,Georgia,73.28
1585,Republic of Moldova,73.26
1621,Russian Federation,73.23


Как можно увидеть по таблице выше на первых местах по продолжительности жизни расположились Прибалтийские страны (Эстония, Латвия, Литва) и Армения. На последних местах Азербайджан, Туркменистан и Таджикистан. Россия в списке находится на 10 месте.
<br><br>
Попробуем понять, что на это влияет. Для начала рассмотрим уровень медицины в этих странах и насколько он различается.
1. Сравним какое кол-во докторов приходится на каждые  на 10 000 населения.
<br>
(т.к. данные по всем странам бывшего СССР есть только за 2014 год, то для сравнения возьмем данные именно за этот год)

In [251]:
datas['md'] = get_stat('medicalDoctors',
                     ['country','period','indicator','result'],                       
                     2014)
datas['md']

,country,result
206,Belarus,50.12
844,Georgia,47.75
1316,Lithuania,42.51
1871,Russian Federation,40.11
1176,Kazakhstan,39.80
138,Azerbaijan,34.46
726,Estonia,33.56
1262,Latvia,31.72
2344,Ukraine,29.92
70,Armenia,28.93


Как можно увидеть из таблицы выше, кол-во врачей прямо не коррелируются с продолжительностью жизни. Единственное совпадение это Туркменистан и Таджикистан с наименьшим кол-вом врачей на 10 т.ч. и с наименьшой продолжительностю жизни среди других стран.
<br>
2. Теперь посмотрим как обстоит дело с медсестринским и акушерским персоналом на 10 000 населения.

In [252]:
datas['nam'] = get_stat('nursingAndMidwife',
                     ['country','period','indicator','result'],                        
                     2014)
datas['nam']

,country,result
2510,Uzbekistan,112.80
200,Belarus,109.80
1361,Lithuania,78.05
1205,Kazakhstan,72.00
2435,Ukraine,66.61
134,Azerbaijan,64.34
772,Estonia,60.72
1263,Kyrgyzstan,59.45
1884,Republic of Moldova,53.11
75,Armenia,51.55


Здесь также нельзя провести прямую связь между продолжительностью жизни и кол-вом мед.персонала.
<br>
Попробуем предположить, что на это влияют различные факторы в стране, такие как чистая питьевая вода, санитария и гигиена.
<br>
3. Посмотрим какой процент населения обеспечены питьевой водой

In [253]:
datas['bdws'] = get_stat('basicDrinkingWaterServices',
                     ['country','period','indicator','result'],                         
                     2017)
datas['bdws']

,country,result
126,Armenia,100.00
1166,Georgia,100.00
3168,Turkmenistan,100.00
1022,Estonia,99.19
1688,Latvia,98.18
3329,Uzbekistan,96.07
270,Belarus,95.98
1778,Lithuania,92.76
2555,Russian Federation,92.62
1580,Kazakhstan,92.31


На первых 3 местах видим Армению, Грузию и Туркменистан со 100% обеспечения населения питьевой водой. Эти цифры вызвают сомнения, т.к. в данных странах присутствуют удаленные населенные пункты, которые сложно обеспечить питьевой водой.
<br>
Также обратимся к отчету Европейского Агентства по окружающей среде, согласно ему:
<br>
<i>"В 2017 году почти 98 % сельского и 100 % городского населения Грузии имели доступ к питьевому водоснабжению, отвечающему требованиям безопасности"</i>
<br>
https://www.eea.europa.eu/ru/publications/vodnye-resursy-kachestvo-poverhnostnyh-vod/
<br><br>
Как можно видеть эти цифры отличаются от того, что мы получили из выборке за 2017 год. Поэтому рассмотрим даные про обеспечение водой за предыдущий год, т.е. за 2016.

In [254]:
datas['bdws'] = get_stat('basicDrinkingWaterServices',
                     ['country','period','indicator','result'],                         
                     2016)
datas['bdws']

,country,result
127,Armenia,100.00
1023,Estonia,99.19
1689,Latvia,98.03
3169,Turkmenistan,97.72
271,Belarus,95.96
3330,Uzbekistan,95.86
1167,Georgia,95.71
2556,Russian Federation,92.14
1779,Lithuania,91.74
3222,Ukraine,91.32


Здесь данные похожи на достоверные и можно провести некую связь между людьми с доступом к питьевой воде и продолжительностю жизни.
<br>
На первых 3 местах мы можем видеть Армению, Эстонию и Латвию. На 2-х последних местах мы видим все те же Азербайджан и Таджикистан.
<br><br>
4. Теперь обратимся к чистым видам топлива и технологиям:

In [255]:
datas['cfat'] = get_stat('cleanFuelAndTech',
                     ['country','indicator','period','result'],
                     2018)
datas['cfat']

,country,result
133,Armenia,95.0
190,Azerbaijan,95.0
285,Belarus,95.0
1045,Estonia,95.0
1634,Kazakhstan,95.0
1748,Latvia,95.0
1805,Lithuania,95.0
2584,Republic of Moldova,95.0
3306,Turkmenistan,95.0
3363,Ukraine,95.0


Здесь мы можем видеть, что 2/3 стран имеют одинаковый процент населения, которое полагается на чистые виды топлива и технологии, т.е. 95%. Если посмотреть данные по этим странам за предыдущие годы, то у них также будет указано 95%. В данном случае мы предполагаем, что это шаблонные данные, которые без изменений добавляли в эту таблицу из года в год и поэтому мы не можем их использовать для дальнейших расчетов.
<br><br>
5. Рассмотрим какой процент населения имеет доступ к базовым санитарным услугам

In [256]:
datas['albss'] = get_stat('atLeastBasicSanitizationServices',
                     ['country','indicator','period','type','result'],                          
                     2017,
                     {'type':'Total'})
datas['albss']

,country,result
9031,Uzbekistan,100.00
2765,Estonia,99.15
8584,Turkmenistan,98.70
4349,Kazakhstan,97.87
628,Belarus,97.79
8194,Tajikistan,97.02
4493,Kyrgyzstan,96.51
8743,Ukraine,96.22
340,Armenia,93.64
4799,Lithuania,93.35


Как можно увидеть, то население всех стран было примерно в равной степени обесечено базовыми санитарными услугами. В данной выборке мы видм явного аутсайдера это Молдавию. Такая разница в данных может говорить или о разнице подсчетов данного параметра в странах или о действительно более плохом положении с санитарными услугами в Молдавии.
<br><br>
Обратимся снова к докладу Европейской коммиссии по окружающей среде и вот, что мы там видим:
<br>
<i>В 2017 году около 17 % сельского населения Молдовы (т.е. 7 % населения страны) не имели доступа к услугам водоснабжения, отвечающим требованиям безопасности, или услугам базового уровня. 31 % сельского населения имел ограниченный доступ к услугам водоотведения и канализации или доступ только к базовой санитарно-гигиенической инфраструктуре (WHO/UNICEF, 2019).</i>
<br>
Мы можем предположить, что эти цифры по Молдавии верны.
<br><br>
6. Теперь обратимся к статистике по обеспечению всеобщего охвата услугами здравоохранения 

In [257]:
datas['uc'] = get_stat('uhcCoverage',
                     ['country','indicator','period','result'],                       
                     2017)
datas['uc']

,country,result
28,Belarus,76.0
170,Kazakhstan,76.0
108,Estonia,75.0
270,Russian Federation,75.0
192,Lithuania,73.0
352,Uzbekistan,73.0
182,Latvia,71.0
178,Kyrgyzstan,70.0
336,Turkmenistan,70.0
12,Armenia,69.0


Здесь мы можем увидеть, что на одном из превых 3 мест находится только Эстония. А среди 3 последних находим Азербайджан. Остальные страны распределились достаточно равномерно по таблице. Единственное, что все же республики с более высокой продолжительностю жизни находятся в верхней половине таблицы, а с низкой продолжительностю находятся в нижней. Только одно исключение это Армения.
<br>
На основании этих данных мы можем предположить, что обеспечение всеобщего охвата услугами здравоохраения косвернно влияет на продолжительсть жизни.
<br><br>
Теперь рассмотрим насколько загрязнение окружающей среды влияет на уроверь смертности.
<br>
7. Рассмотрим уровень смертности от загрязнений атмосферного и домашнего воздуха на 100 000 человек
<br>
(возьмем полную статистику про оба пола, 2016 год и со стандартизацией по возрасту)

In [260]:
datas['apdr'] = get_stat('airPollutionDeathRate',
                        ['country','type','indicator','period','gender','result'],                       
                        2016,
                        {'type':'Total', 'gender':'Both sexes',\
                         'indicator':'Ambient and household air pollution attributable death rate (per 100 000 population, age-standardized)'})
datas['apdr']

,country,result
5763,Tajikistan,129.30
3207,Kyrgyzstan,110.70
2235,Georgia,101.80
6339,Uzbekistan,81.14
6051,Turkmenistan,79.30
4755,Republic of Moldova,78.28
6123,Ukraine,70.72
327,Azerbaijan,63.95
3063,Kazakhstan,62.68
507,Belarus,60.74


Как можно увидеть по данной статистике, что последние 3 места по смертности от загрязнения окружающей среды занимают прибалтийские страны, что может говорить о том, что чистая окружающей среда может напрямую влиять на продолжительность жизни.
<br><br>
8. Рассмотрим как обстоит дело с употреблением табака в рассматриваемых нами странах

In [262]:
datas['ta'] = get_stat('tobaccoAge15',
                     ['country','indicator','period','gender','result'],                       
                     2018,
                     {'gender':'Both sexes'})
datas['ta']

,country,result
1998,Latvia,36.7
1188,Estonia,30.5
1377,Georgia,29.7
3024,Russian Federation,28.3
1944,Kyrgyzstan,27.9
2106,Lithuania,27.1
108,Armenia,26.7
324,Belarus,26.6
3726,Ukraine,25.5
2970,Republic of Moldova,25.3


Данная статистика приводится только по части нужных нам стран (13 из 15), поэтому попробуем сделать предположения исходя из той информации, что у нас есть. Как можно видеть из таблицы на первых местах по потреблению табака находятся Латви, Эстония и Грузия, а на последних Казахстан, Азербайджан и Узбекистан. По этой информации мы не можем однозначно сказать насколько влияет потребление табака на продолжительность жизни, т.к. здесь видно противорече между продолжительносю жизни и кол-вом курящих людей.
<br><br>
9. Рассмотрим употребление алкоголя на душу населения

In [263]:
datas['asa'] = get_stat('AlcoholSubstanceAbuse',
                     ['country','period','indicator','gender','result'],                       
                     2018,
                     {'gender':'Both sexes'})
datas['asa']

,country,result
1458,Lithuania,13.22
1383,Latvia,12.77
219,Belarus,11.45
2022,Republic of Moldova,11.43
2067,Russian Federation,11.19
828,Estonia,9.23
2592,Ukraine,8.32
948,Georgia,8.25
1353,Kyrgyzstan,6.26
99,Armenia,5.55


Здесь, как мы можем видеть, что употребление алкоголя не оказывает значительного влияние на продолжительность жизни. Страны с большим процентом потребления алкоголя на душу населеничя при этом имеют и большую продолжительность жизни. Из чего мы можем сделать предварительный вывод, что алкоголь существенно не влияет на продолжительность жизни в среднем по стране.
<br><br>
10. Обратим наше внимание на ситуацию с дорожно-транспортными происшествиями в исследуемых нами странах. Рассмотрим кол-во смертей в дтп на 100 тыс. человек.

In [264]:
datas['rtd'] = get_stat('roadTrafficDeaths',
                     ['country','indicator','period','result'],                       
                     2016)
datas['rtd']

,country,result
154,Tajikistan,18.1
130,Russian Federation,18.0
80,Kazakhstan,17.6
5,Armenia,17.1
84,Kyrgyzstan,15.4
58,Georgia,15.3
162,Turkmenistan,14.5
164,Ukraine,13.7
170,Uzbekistan,11.5
127,Republic of Moldova,9.7


Вот здесь мы видим некоторую связь между странами с высокой продолжительностю жизни и гибелью людей в дтп. Прибалтийские страны в самой высокой продожительностю жизни находятся в самом низу рейтинга, но аутсайдеры рейтина продолжительности жизни распределены по указанной выше таблице - рейтинг возглавляет Таджикистан, но при этом Туркменестан и Азейрбаджан присутствуют в середине и почти в конце списка. Мы можем предположить, что дтп на дорогах влияют на продолжительность жизни, но в таких странах как Туркменестан, Азербайджан и Узбекистан присутствуют дополнительны факторы влиящие на жизнь и здоровье людей.
<br><br>
Теперь обратим наше внимание на данные по уровням репродуктивного здоровья населения стран:
<br>
11. Рассмотрим коэффициент подростковой рождаемости на 1000 женщин в возрасте 15-19 лет

In [265]:
datas['ab'] = get_stat('adolescentBirthRate',
                     ['country','period','indicator','result'],                       
                     2016)
datas['ab']

,country,result
1936,Tajikistan,54.3
150,Azerbaijan,52.8
726,Georgia,43.6
1052,Kyrgyzstan,38.1
1006,Kazakhstan,28.1
96,Armenia,24.5
1597,Republic of Moldova,23.8
2036,Ukraine,23.2
1650,Russian Federation,21.5
2129,Uzbekistan,20.9


Здесь можн увидеть, что в странах находящихся в европейской части меньше кол-во подросткой рождаемости. Это можно объяснить традициями данных стран. Например если взять статистику за 2017 год по среднему возрасту рождения первенца (https://lv.sputniknews.ru/infographics/20190418/11360511/vozrast-zhenschiny-rozhdenie-perventsa.html), то мы можем увидеть, что средний возраст женщин, которые становятся матерями впервые, в Европе постоянно растет. Далее использовать данную выборку мы не будем, т.к. она показывает только часть данных, а не всю картинку по стране.
<br><br>
12. Теперь рассмотрим вероятность смерти между рождением и 1 годом жизни ребенка на 1000 живорождений

In [287]:
datas['imr'] = get_stat('infantMortalityRate',
                        ['country','period','indicator','gender','result'],                       
                        2019,
                        {'gender':'Both sexes'})
datas['imr']

,country,result
29589,Tajikistan,29.60
1638,Azerbaijan,18.24
16209,Kyrgyzstan,16.36
24582,Republic of Moldova,12.35
1086,Armenia,10.51
15504,Kazakhstan,9.30
11052,Georgia,8.53
25041,Russian Federation,4.93
16470,Latvia,3.09
17364,Lithuania,2.99


Здесь мы можем увидеть достаточно высокую детскую смертность у аутсайдеров нашего рейтинга и достаточно низкую у прибалтийских стран и Белоруссии. Примерно такой же результат мы можем получить, если рассмотрим данные из таблицы neonatalMortalityRate - вероятность смерти детей в первые 28 дней жизни и under5MortalityRate - вероятность смерти детей в возрасте до 5 лет на 1000 живорождений. Из чего мы можем сделать вывод, что уровень перинатальной медицины и педиатрии в странах с наименьшем длительностью жизни находится на более худшем уровне, чем в странах с большей продолжительностю жизни.
<br><br>
Теперь рассмотрим уровень заболеваний в разных странах:
<br>
13. Начнем с вероятность смерти в возрасте от 30 до 70 лет от любого из сердечно-сосудистых заболеваний, рака, диабета или хронических респираторных заболеваний.

In [291]:
datas['cce'] = get_stat('30-70cancerChdEtc',
                        ['country','period','indicator','gender','result'],                       
                        2016,
                        {'gender':'Both sexes'})
datas['cce']

,country,result
2520,Turkmenistan,29.5
1275,Kazakhstan,26.8
2025,Russian Federation,25.4
2400,Tajikistan,25.3
930,Georgia,24.9
1335,Kyrgyzstan,24.9
1980,Republic of Moldova,24.9
2550,Ukraine,24.7
2640,Uzbekistan,24.5
210,Belarus,23.7


Как можно видеть выше, то прибалтийские страны показывают наименьший процент смертности от заболеваний. Также как и в предыдущем случае можем предположить, что уровень медицины в этих странах позволяет предотвращать смертность по этим заболеваниям.
<br><br>
14. Раззметрим кол-во заболеваний туберкулезом на 100 тыс. человек в год

In [307]:
datas['iot'] = get_stat('incedenceOfTuberculosis',
                        ['country','indicator','period','result'],                       
                        2018)
datas['iot']

,country,result
1841,Kyrgyzstan,116.0
2796,Republic of Moldova,86.0
3380,Tajikistan,84.0
1301,Georgia,80.0
3598,Ukraine,80.0
3718,Uzbekistan,70.0
1761,Kazakhstan,67.0
201,Azerbaijan,63.0
2856,Russian Federation,54.0
3538,Turkmenistan,46.0


Это достаточно важный показатель, которые может отображать ситуацию со здравоохранением в стране. Согласно докладу ВОЗ о Туберкулезе (https://www.who.int/ru/news-room/fact-sheets/detail/tuberculosis) - "во всем мире туберкулез является одной из 10 основных причин смерти и основной причиной смерти (опережая ВИЧ/СПИД)... риску подвергаются все возрастные группы. Более 95% случаев заболевания и смерти происходят в развивающихся странах." Эта информация как раз является достаточно хорошим показателем, что чем лучше уровень медицины в стране, тем меньше в ней болеющих туберкулезом людей.
<br><br>
15. В завершении просмотра аналитических данных сравним общий уровень самоубийств на 100 тыс. населения

In [337]:
datas['csr'] = get_stat('crudeSuicideRates',
                       ['country','period','indicator','gender','result'],                       
                       2015,
                       {'gender':'Both sexes'})
datas['csr']

,country,result
1443,Lithuania,34.9
2028,Russian Federation,32.1
213,Belarus,26.3
1278,Kazakhstan,24.9
2553,Ukraine,23.1
1368,Latvia,21.6
1983,Republic of Moldova,16.4
813,Estonia,15.8
933,Georgia,8.1
1338,Kyrgyzstan,8.1


Данная информация не оказывает большого влияния на общий уровень жизни в стране, т.к. данные цифры представляют собой достаточно маленький процент от общего населения, но все же здесь мы можем видеть, в Литве, России и Белоруссии достаточно высокий уровень самоубийств. К сожалению в данной выборке есть информация только за 2015 год, но если обратиться к данным ВОЗ (https://lv.sputniknews.ru/world/20190918/12464844/Latviya---sredi-liderov-ES-po-chislu-samoubiystv.html), то мы можем увидеть, что Латвия вышла на первое место по числу самоубийств в странах ЕС. "Самые высокие показатели самоубийств отмечаются в странах с высоким уровнем дохода, где среди молодежи суицид занимает вторую строчку в списке основных причин смерти", — сообщает ВОЗ. Так что, как это странно не звучано, но высокий уровень дохода и жизни взаимосвязан с высоким уровнем самоубийств в стране.